In [1]:
# !pip install opencv-python scikit-learn scikit-image matplotlib spectral keras_tuner vis

In [2]:
import tensorflow as tf

In [3]:
from tensorflow import keras as keras
from keras import layers as layers
import numpy as np

In [4]:
import os, timeit
import cv2
from skimage.filters import threshold_otsu
import matplotlib.pyplot as plt
from math import inf as inf
import numpy as np

In [5]:
from spectral.io import envi as envi
from spectral import imshow

In [6]:
from sklearn.decomposition import IncrementalPCA

In [7]:
import keras_tuner as kt
import sys

In [8]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  0


In [9]:
from sys import platform
DATA_DIRECTORY = ""
SLASH = ""
if platform == "linux" or platform == "linux2":
    DATA_DIRECTORY = "/home/nitintyagi/wheat data/BULK/"
    SLASH = "/"
elif platform == "win32":
    DATA_DIRECTORY = "D:\mvl\wheat\data\BULK\\"
    SLASH="\\"

In [10]:
#Testing Constants
TESTING = False

#Constants
BAND_NUMBER = 60
FILLED_AREA_RATIO = 0.90
IMAGE_COUNT = int(200/4)
NUM_VARIETIES = 4
NUM_OF_BANDS = 20
FIRST_BAND = 21
LAST_BAND = 149

IMAGE_WIDTH = 20
IMAGE_HEIGHT = 20

NUM_EPOCHS = 40
ACTIVATION_TYPE =  ["relu", "tanh","sigmoid"]
BATCH_SIZE = 2*NUM_VARIETIES

LEARNING_RATE_BASE = 0.0001
MIN_LEARNING_RATE_BASE = LEARNING_RATE_BASE/10

FACTOR = 3
NUM_MODELS = 3

In [11]:
def result_file_name():
    return "NumVar_"+str(NUM_VARIETIES)+"_ImageCount_"+str(IMAGE_COUNT)+"_Factor_"+str(FACTOR)+"_MinLR_"+str(MIN_LEARNING_RATE_BASE)+"_LR_"+str(LEARNING_RATE_BASE)+"_FilledArea_"+str(FILLED_AREA_RATIO)+"_NumOfBands_"+str(NUM_OF_BANDS)+"_FB_"+str(FIRST_BAND)+"_LB_"+str(LAST_BAND)+"_BandNo_"+str(BAND_NUMBER)+"_ImageHeight_"+str(IMAGE_HEIGHT)+"_ImageWidth_"+str(IMAGE_WIDTH)+"_BatchSize_"+str(BATCH_SIZE)


In [12]:
def start_timer():
    print("Testing started")
    return timeit.default_timer()

def end_timer():
    return timeit.default_timer()

def show_time(tic,toc): 
    test_time = toc - tic
    print('Testing time (s) = ' + str(test_time) + '\n')

In [13]:
def exactPathHDR(variety,file):
    return DATA_DIRECTORY+variety+SLASH+file+".bil.hdr"

def exactPathBIL(variety,file):
    return DATA_DIRECTORY+variety+SLASH+file+".bil"

In [14]:
def getROI(img,band_number):
    img_band = img.read_band(band_number)
    threshold = threshold_otsu(img_band)
    roi=[]
    for x in range(img_band.shape[0]):
        a=[]
        for y in range(img_band.shape[1]):
            if img_band[x][y]>threshold:
                a.append(1)
            else:
                a.append(0)
        roi.append(a)
    return roi

In [15]:
#Returns range for x and y from where we have to crop images
def getRangeXandY(img,band_number):
    img_band = img.read_band(band_number)
    roi = getROI(img,band_number)
    xmin = inf
    xmax = 0
    ymin = inf
    ymax = 0
    for x in range(img_band.shape[0]):
        for y in range(img_band.shape[1]):
            if roi[x][y]==1:
                if x<xmin:
                    xmin=x
                if x>xmax:
                    xmax=x
                if y<ymin:
                    ymin=y
                if y>ymax:
                    ymax=y
    return xmin, xmax, ymin, ymax

In [16]:
def getCroppedImage(img,band_number):
    xmin, xmax, ymin, ymax = getRangeXandY(img,band_number)
    new_img = img[xmin:xmax, ymin:ymax, :]
    return new_img    

In [17]:
def getCroppedROI(img,band_number):
    xmin, xmax, ymin, ymax = getRangeXandY(img,band_number)
    roi = np.array(getROI(img,band_number))
    roi = roi[xmin:xmax, ymin:ymax]
    return roi   

In [18]:
def getUsefulImage(img,band_number):
    crop_img = getCroppedImage(img,band_number)
    crop_roi = getCroppedROI(img,band_number)
    for x in range(crop_img.shape[2]):
        band = crop_img[:,:,x]
        crop_img[:,:,x] = band*crop_roi
    return crop_img

In [19]:
data_augmentation = keras.Sequential([
    layers.RandomCrop(height=IMAGE_HEIGHT, width=IMAGE_WIDTH),
    layers.RandomRotation(factor=(-0.1, 0.1)),
    layers.RandomZoom(height_factor=(-0.1, 0.1), width_factor=(-0.1,0.1)),
    layers.RandomFlip(mode="horizontal_and_vertical", seed=None)
])

def getAugumentedImage(img,band_number):
    new_img = getUsefulImage(img,band_number)
    augmented_image = data_augmentation(new_img) 
    return augmented_image

def checkAugumentedImage(augmented_image):
    aug_band = augmented_image[:,:,0]
    filled_area_ratio = (np.count_nonzero(aug_band))/(aug_band.shape[0]*aug_band.shape[1])
    if filled_area_ratio > FILLED_AREA_RATIO :
        return True
    else:
        return False

In [20]:
## Dimensional Reduction Method
def DL_Method(HSI, numComponents = NUM_OF_BANDS):
    RHSI = np.reshape(HSI, (-1, HSI.shape[2]))
    n_batches = 10
    inc_pca = IncrementalPCA(n_components=numComponents)
    for X_batch in np.array_split(RHSI, n_batches):
        inc_pca.partial_fit(X_batch)
    X_ipca = inc_pca.transform(RHSI)
    RHSI = np.reshape(X_ipca, (HSI.shape[0],HSI.shape[1], numComponents))
    return RHSI

In [21]:
# List for All varieties
VARIETIES = []
VARIETIES_CODE = {}

for name in os.listdir(DATA_DIRECTORY):
    if (name.endswith(".hdr") or name.endswith(".bil")):
        continue
    VARIETIES_CODE[name] = len(VARIETIES)
    VARIETIES.append(name)
    if len(VARIETIES)==NUM_VARIETIES:
        break

In [22]:
#List for all file names in varities
FILES = []
MAX_FILE_NUM = 4
for x in range(1,MAX_FILE_NUM+1):
    FILES.append("B_"+str(x))

In [23]:
#List of all images
images = []
images_label = []
for v in VARIETIES:
    for f in FILES:
        try:
            img = envi.open(exactPathHDR(v,f),exactPathBIL(v,f))
            images.append(img)
            images_label.append(v)
        except:
            pass

In [24]:
train_dataset = []
train_dataset_label = []
test_dataset = []
test_dataset_label = []

In [25]:
tic = start_timer()
for index, img in enumerate(images):
    count = 0
    label = images_label[index]
    while count<IMAGE_COUNT:
        aug_img = getAugumentedImage(img,BAND_NUMBER)
        
        if checkAugumentedImage(aug_img):
            aug_img = DL_Method(aug_img[:,:,FIRST_BAND:LAST_BAND+1])
            if count%5 == 0:
                test_dataset.append(aug_img)
                test_dataset_label.append(label)
            else:
                train_dataset.append(aug_img)
                train_dataset_label.append(label)
            count+=1  
            
    if TESTING:
        break
        
toc = end_timer()
show_time(tic,toc)

Testing started
Testing time (s) = 1547.94772679999



In [26]:
train_dataset = np.array(train_dataset)
train_dataset_label = np.array([VARIETIES_CODE[label] for label in train_dataset_label])
test_dataset = np.array(test_dataset)
test_dataset_label = np.array([VARIETIES_CODE[label] for label in test_dataset_label])

In [27]:
for index,data in enumerate(test_dataset):
#     imshow(data)
    print(test_dataset_label[index])

0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3


In [28]:
for index,data in enumerate(train_dataset):
#     imshow(data)
    print(train_dataset_label[index])

0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3


In [29]:
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import pyplot as plt
# import math, sys, pdb, os
import keras
import keras.backend as K
import tensorflow as tf
from keras.layers import Input, Conv2D, MaxPooling2D, Activation, BatchNormalization, Add, Conv2DTranspose, Flatten, Dense, Conv1D, AveragePooling2D, LeakyReLU, PReLU, GlobalAveragePooling2D
from keras.layers.core import Dropout
from keras.layers import concatenate
from keras.models import Model

import os, pdb, timeit
import numpy as np
from keras.callbacks import TensorBoard
from keras.optimizers import Adam
import matplotlib
matplotlib.use('Agg')
from matplotlib import pyplot as plt

from sklearn.metrics import confusion_matrix, precision_recall_fscore_support
import matplotlib.cm as cm
import cv2
from keras import activations
import vis
# from vis.visualization import visualize_saliency, overlay
# from vis.utils import utils

In [30]:
def normalizeDataWholeSeed(data,normalization_type='max'):
    
    if normalization_type == 'max':
        for idx in range(data.shape[0]):
            data[idx,:,:,:] = data[idx,:,:,:]/np.max(abs(data[idx,:,:,:]))
            
    elif normalization_type == 'l2norm':
        from numpy import linalg as LA
        for idx in range(data.shape[0]):
            data[idx,:,:,:] = data[idx,:,:,:]/LA.norm(data[idx,:,:,:]) # L2-norm by default        
        
    return data

In [31]:
def make_hyperparam_string(learning_rate_base, batch_size, kernel_size, dropout_rate, num_training,
                           num_nodes_fc, activation_type):
    hparam = ""
    hparam += str(num_nodes_fc) + "nodes_" + str(learning_rate_base) + "lr_" + str(batch_size) + "batch_" + str(
        kernel_size) + "kernel_" + str(dropout_rate) + "drop_" + str(
        num_training) + "train_" + activation_type

    return hparam

In [32]:
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):

    import itertools
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.clim(0,sum(cm[0,:]))
    plt.xlabel('Predicted label')

In [33]:
def top_K_classification_accuracy(y_predicted, y_true, K=1):

    num_samples = y_predicted.shape[0]
    num_classes = y_predicted.shape[1]

    if K > num_classes:
        sys.exit(1)

    temp = np.zeros((num_samples,))

    for idx in range(num_samples):
        curr_predicted = np.argsort(y_predicted[idx,:])
        curr_predicted = curr_predicted[::-1] # descending

        if y_true[idx] in curr_predicted[:K]:
            temp[idx] = 1

    return 100.0 * np.sum(temp)/num_samples

In [34]:
def conv2D_ResNet(x, kernel_size, activation_type, dropout_rate, num_filters_first_conv1D):

    x_orig = x

    # Batch norm
    x = BatchNormalization()(x)

    # 1x1 Conv2D
    x = Conv2D(num_filters_first_conv1D, kernel_size=1, activation=None, use_bias=False, padding='same',
               kernel_initializer='truncated_normal')(x)

    # Activation
    x = Activation(activation_type)(x)

    x = BatchNormalization()(x)

    # 3x3 Conv2D
    x = Conv2D(num_filters_first_conv1D, kernel_size, activation=None, use_bias=True, padding='same',
               kernel_initializer='truncated_normal')(x)

    # Activation       
    x = Activation(activation_type)(x)

    x = BatchNormalization()(x)

    # 1x1 Conv2D
    x = Conv2D(num_filters_first_conv1D*4, kernel_size=1, activation=None, use_bias=False, padding='same',
               kernel_initializer='truncated_normal')(x)

    # Skip connection
#     if int(x.shape[3]) != int(x_orig.shape[3]):
#         x_orig = Conv2D(int(x.shape[3]), kernel_size=1, activation=None, use_bias=False, padding='same',
#                kernel_initializer='truncated_normal')(x_orig)

    # Activation      
    x = Activation(activation_type)(x)

#     x = Add()([x, x_orig])

    # Dropout
    return Dropout(dropout_rate)(x)

In [35]:
def createBlock_ResNet2D(x, num_layers, kernel_size, activation_type, dropout_rate, num_filters_first_conv1D):

    for idx_layer in range(num_layers):
        x = conv2D_ResNet(x, kernel_size, activation_type, dropout_rate, num_filters_first_conv1D)

    return x

In [36]:
# growth_rate: number of filters for each normal convolution ('k' in the paper)
def ResNet2D_classifier(data_num_rows, data_num_cols, num_classes, kernel_size=3, num_layers_each_block=[6, 12, 24, 16],
                        num_chan_per_block = [64,128,256,512], activation_type=['relu'], dropout_rate=0.0, num_input_chans=1, num_nodes_fc=64):

    input_data = Input(shape=(data_num_rows, data_num_cols, num_input_chans))

    # Input layer: Conv2D -> activation
    x = Conv2D(num_chan_per_block[0], kernel_size, activation=None, use_bias=True, padding='same',
               kernel_initializer='truncated_normal')(input_data)

    # Activation
    x = Activation(activation_type)(x)


    #  Blocks & Downsampling Layers
    for idx_block in range(len(num_layers_each_block)):
        x = createBlock_ResNet2D(x, num_layers_each_block[idx_block], kernel_size, activation_type, dropout_rate,
                                 num_chan_per_block[idx_block])

        x = BatchNormalization()(x)

        if idx_block != len(num_layers_each_block)-1:
            x = Conv2D(num_chan_per_block[idx_block]*2, kernel_size, strides = 2, activation=None, use_bias=True, padding='valid',
                   kernel_initializer='truncated_normal')(x)
        else:
            x = GlobalAveragePooling2D()(x)

        x = Dropout(dropout_rate)(x)

    # Output layer
    x = BatchNormalization()(x)
    x = Dense(units=num_nodes_fc, activation=None, kernel_initializer='truncated_normal')(x)

    # Activation
    x = Activation(activation_type)(x)

    x = BatchNormalization()(x)
    output_data = Dense(units=num_classes, activation='softmax', kernel_initializer='truncated_normal')(x)

    return Model(inputs=input_data, outputs=output_data)

In [37]:
def evaluate(model,dataset,dataset_label,normalization_type):
    print("--------------Make Predictions--------------")    
    x = np.array(dataset)
    labels = np.array(dataset_label)
    
    # Normalize the data
    x = normalizeDataWholeSeed(x,normalization_type=normalization_type)
    
    num = x.shape[0]

    tic = start_timer()
    labels_predicted = model.predict(x)
    toc = end_timer()
    show_time(tic,toc)
    
    print(labels_predicted)
    print("--------")
    # Classification accuracy
    labels_integer_format = labels
    labels_predicted_integer_format = np.argmax(labels_predicted, axis=1)

    acc_top2 = top_K_classification_accuracy(labels_predicted, labels_integer_format, K=2)
    acc_top1 = top_K_classification_accuracy(labels_predicted, labels_integer_format, K=1)
    
    # Confusion matrices
    confusion_matrix_results = confusion_matrix(labels_integer_format, labels_predicted_integer_format)
    print("Confusion matrix = ")
    print(confusion_matrix_results)
    print("------------------------------------------------")

In [38]:
def predict(model,normalization_type):
    evaluate(model,train_dataset,train_dataset_label,normalization_type)
    
    evaluate(model,test_dataset,test_dataset_label,normalization_type)
    
    
    # Precision, Recall, F1
#     macro_avg = np.asarray(
#         precision_recall_fscore_support(labels_test_integer_format, labels_predicted_test_integer_format,
#                                         average='macro'))
#     macro_avg_precision = macro_avg[0]
#     macro_avg_recall = macro_avg[1]
#     macro_avg_fscore = macro_avg[2]

#     print('Top-1 accuracy (%) = ' + str(acc_top1) + '\n')
#     print('Top-2 accuracy (%) = ' + str(acc_top2) + '\n')
#     print('Macro-avg precision = ' + str(macro_avg_precision) + '\n')
#     print('Macro-avg recall = ' + str(macro_avg_recall) + '\n')
#     print('Macro-avg f-score = ' + str(macro_avg_fscore) + '\n')

#     print("--------------Done--------------")

#     print("--------------Compute Saliency Maps--------------")
#     results_test_dir = os.path.join(results_dir, 'test')
#     if not os.path.exists(results_test_dir):
#         os.makedirs(results_test_dir)

#     # Swap softmax with linear
#     model.layers[-1].activation = activations.linear
#     model = utils.apply_modifications(model)

#     for idx_wheat in range(num_test):

#         grads = visualize_saliency(model, layer_idx=-1, filter_indices=np.argmax(labels_test[idx_wheat, :], axis=0),
#                                    seed_input=x_test[idx_wheat], backprop_modifier=None)

#         ss_img = np.sqrt(np.sum(abs(x_test[idx_wheat, :, :, :]) ** 2, axis=2))
#         ss_img /= np.max(ss_img)

#         plt.figure(1)
#         plt.subplot(3, 1, 1)
#         plt.imshow(ss_img, cmap='gray')
#         plt.clim(0, 1)
#         plt.axis('off')
#         plt.colorbar()

#         plt.subplot(3, 1, 2)
#         plt.imshow((grads * np.uint8(255)).astype('uint8'), cmap='jet')
#         plt.clim(0, 255)
#         plt.axis('off')
#         plt.colorbar()

#         jet_heatmap = np.uint8(cm.jet(grads)[..., :3] * np.uint8(255))

#         plt.subplot(3, 1, 3)
#         ss_img = cv2.cvtColor((ss_img * np.uint8(255)).astype('uint8'), cv2.COLOR_GRAY2RGB)
#         plt.imshow(overlay(jet_heatmap, ss_img, alpha=0.3))
#         plt.clim(0, 255)
#         plt.axis('off')
#         plt.colorbar()

#         plt.savefig(os.path.join(results_test_dir, str(idx_wheat+1) + '.png'))
#         plt.clf()

#     print("--------------Done--------------")

#     print("--------------Save the information--------------")

#     # Write some information to files
#     f = open(os.path.join(results_test_dir, 'testing_info.txt'), 'w')
#     f.write("Wheat types = " + str(wheat_types) + "\n")
#     f.write("Confusion matrix \n")
#     f.write(str(confusion_matrix_results) + "\n")
#     f.write("Normalization type = " + str(normalization_type) + "\n")
#     f.write("# test samples = %d \n" % (num_test))
#     f.write("Top-1 test accuracy = %f \n" % (acc_top1))
#     f.write("Top-2 test accuracy = %f \n" % (acc_top2))
#     f.write("Macro-avg precision = %f \n" % (macro_avg_precision))
#     f.write("Macro-avg recall = %f \n" % (macro_avg_recall))
#     f.write("Macro-avg f-score = %f \n" % (macro_avg_fscore))
#     f.write("Test time (s) = " + str(test_time) + "\n")
#     f.close()

#     # Save confusion matrices
#     plt.figure(1)
#     plot_confusion_matrix(confusion_matrix_results, classes=wheat_types, normalize=False, title='Confusion matrix')
#     plt.savefig(os.path.join(results_test_dir,'confusionMatrix.png'))
#     plt.clf()

#     print("--------------Done--------------")

#     print("--------------Save the information for the training phase--------------")
    
#     import pandas as pd
    
#     # Save the trained model
#     model.save_weights(os.path.join(results_dir, 'trainedResNetB_weights.h5'))
    
#     # Extract the training loss   
#     training_loss = hist.history['loss']

#     # Save the training loss
#     df = pd.DataFrame(data={'training loss': training_loss},index=np.arange(num_epochs)+1)
#     df.to_csv(os.path.join(results_dir,'training_loss.csv'))
    
#     # Save the training loss as a figure
#     plt.figure(1)
#     plt.title('Loss')
#     plt.plot(training_loss, color='b',label='Training')
#     plt.legend()
#     plt.grid()
#     plt.savefig(os.path.join(results_dir,'training_loss.png'))
#     plt.clf()   
    
#     # Write a file with general information
#     f = open(os.path.join(results_dir,'training_info.txt'),'w')
#     f.write(hparams + '\n')
#     f.write('Wheat types = ' + str(wheat_types)+'\n')
#     f.write('Training time (s) = %f \n' %(training_time))
#     f.write('Normalization type = ' + str(normalization_type)+ '\n')
#     f.write('# epochs = ' + str(num_epochs) + '\n')
#     f.write('# training samples = %d \n' %(num_training))
#     f.close()
    
#     print("--------------Done--------------")

In [39]:
def createAndTrainResNetB():
    
    learning_rate_base = LEARNING_RATE_BASE
    kernel_size = 3
    num_epochs = NUM_EPOCHS
    batch_size = BATCH_SIZE
    dropout_rate = 0.15 
    activation_type = 'relu'
    num_nodes_fc = 256
    wheat_types =  VARIETIES
    normalization_type = 'max'
    num_layers_each_block = [1, 0, 0, 0]
    num_chan_per_block = [128, 128, 256, 256]
    N_classes = len(wheat_types)
    
    ############ Load data ############
    print("--------------Load Data--------------")

    # Load training data and their corresponding labels
    x_training = np.array(train_dataset)
    labels_training = np.array(train_dataset_label)
    
    # Normalize the data
    x_training = normalizeDataWholeSeed(x_training,normalization_type=normalization_type)
    
    # Extract some information
    num_training = x_training.shape[0]
    N_spatial = x_training.shape[1:3]
    N_bands = x_training.shape[3]
    
    print('#training = %d' %(num_training))
    
    print("--------------Done--------------")
    
    
    ############ Prepare the path for saving the models/stats ############
#     print("--------------Prepare a path for saving the models/stats--------------")
    
#     hparams = make_hyperparam_string(learning_rate_base, batch_size, kernel_size, dropout_rate,
#                                      num_training, num_nodes_fc, activation_type)
#     print('Saving the model to...')
    
#     results_dir = os.path.join('./results/',hparams)
    
#     if not os.path.exists(results_dir):
#         os.makedirs(results_dir)
#     print(results_dir)

#     print("--------------Done--------------")

    ############ Create a model ############
    print("--------------Create a model--------------")
    
    # Generate a model
    model = ResNet2D_classifier(data_num_rows=N_spatial[0], data_num_cols=N_spatial[1], num_classes=N_classes,
                                kernel_size=kernel_size, num_layers_each_block=num_layers_each_block,
                                num_chan_per_block=num_chan_per_block, activation_type=activation_type,
                                dropout_rate=dropout_rate, num_input_chans=N_bands, num_nodes_fc=num_nodes_fc)

    # Compile the model
    adam_opt = Adam(learning_rate=LEARNING_RATE_BASE, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.01)
    model.compile(loss='sparse_categorical_crossentropy', optimizer=adam_opt, metrics=['accuracy'])

    # Create a Tensorboard callback
#     tbCallBack = TensorBoard(log_dir=results_dir, histogram_freq=0, write_graph=False, write_images=False)
    
    print("--------------Done--------------")

    ############ Train the model ############
    print("--------------Begin training the model--------------")

#     tic = timeit.default_timer()
    
    # Train the model
#     hist = model.fit(x=x_training,y=labels_training,batch_size=batch_size,  epochs = num_epochs, initial_epoch = 0, verbose=2, callbacks = [tbCallBack],validation_split=0.15,shuffle=True)

#     toc = timeit.default_timer()
#     training_time = toc-tic
#     print("Total training time = " + str(training_time))
    
    print("--------------Done--------------")

    return model

In [40]:
model = createAndTrainResNetB()

--------------Load Data--------------
#training = 640
--------------Done--------------
--------------Create a model--------------
--------------Done--------------
--------------Begin training the model--------------
--------------Done--------------


In [41]:
tic = start_timer()
model.fit(x=train_dataset,y=train_dataset_label,batch_size=BATCH_SIZE, epochs=20, initial_epoch = 0, verbose=2,validation_split=0.40,shuffle=True)
toc = end_timer()
show_time(tic,toc)

Testing started
Epoch 1/20
48/48 - 13s - loss: 1.5024 - accuracy: 0.2839 - val_loss: 1.4151 - val_accuracy: 0.0000e+00 - 13s/epoch - 270ms/step
Epoch 2/20
48/48 - 11s - loss: 1.2626 - accuracy: 0.4323 - val_loss: 1.3439 - val_accuracy: 0.0156 - 11s/epoch - 222ms/step
Epoch 3/20
48/48 - 11s - loss: 1.2117 - accuracy: 0.4661 - val_loss: 1.3251 - val_accuracy: 0.0547 - 11s/epoch - 227ms/step
Epoch 4/20
48/48 - 11s - loss: 1.1247 - accuracy: 0.5156 - val_loss: 1.3453 - val_accuracy: 0.0430 - 11s/epoch - 235ms/step
Epoch 5/20
48/48 - 11s - loss: 1.0098 - accuracy: 0.6198 - val_loss: 1.3382 - val_accuracy: 0.0781 - 11s/epoch - 235ms/step
Epoch 6/20
48/48 - 11s - loss: 0.9302 - accuracy: 0.6745 - val_loss: 1.3593 - val_accuracy: 0.0938 - 11s/epoch - 235ms/step
Epoch 7/20
48/48 - 12s - loss: 0.9067 - accuracy: 0.6849 - val_loss: 1.3972 - val_accuracy: 0.0977 - 12s/epoch - 243ms/step
Epoch 8/20
48/48 - 12s - loss: 0.9118 - accuracy: 0.6641 - val_loss: 1.4084 - val_accuracy: 0.1367 - 12s/epoch -

In [42]:
model.evaluate(test_dataset,test_dataset_label)

5/5 [==============================] - 1s 140ms/step - loss: 1.4963 - accuracy: 0.3125


[1.496333360671997, 0.3125]

In [43]:
model.evaluate(train_dataset,train_dataset_label)

20/20 [==============================] - 3s 141ms/step - loss: 0.8659 - accuracy: 0.6625


[0.8658636808395386, 0.6625000238418579]

In [44]:
model = createAndTrainResNetB()

--------------Load Data--------------
#training = 640
--------------Done--------------
--------------Create a model--------------
--------------Done--------------
--------------Begin training the model--------------
--------------Done--------------


In [45]:
tic = start_timer()
model.fit(x=train_dataset,y=train_dataset_label,batch_size=BATCH_SIZE, epochs=40, initial_epoch = 0, verbose=2,validation_split=0.40,shuffle=True)
toc = end_timer()
show_time(tic,toc)

Testing started
Epoch 1/40
48/48 - 13s - loss: 1.5571 - accuracy: 0.2578 - val_loss: 1.3996 - val_accuracy: 0.4609 - 13s/epoch - 273ms/step
Epoch 2/40
48/48 - 11s - loss: 1.2975 - accuracy: 0.4036 - val_loss: 1.4221 - val_accuracy: 0.0508 - 11s/epoch - 237ms/step
Epoch 3/40
48/48 - 11s - loss: 1.1985 - accuracy: 0.4870 - val_loss: 1.4567 - val_accuracy: 0.0117 - 11s/epoch - 237ms/step
Epoch 4/40
48/48 - 11s - loss: 1.0985 - accuracy: 0.5469 - val_loss: 1.4762 - val_accuracy: 0.0078 - 11s/epoch - 236ms/step
Epoch 5/40
48/48 - 11s - loss: 1.0326 - accuracy: 0.6094 - val_loss: 1.5060 - val_accuracy: 0.0000e+00 - 11s/epoch - 236ms/step
Epoch 6/40
48/48 - 11s - loss: 0.9560 - accuracy: 0.6615 - val_loss: 1.5311 - val_accuracy: 0.0039 - 11s/epoch - 237ms/step
Epoch 7/40
48/48 - 11s - loss: 0.9295 - accuracy: 0.6849 - val_loss: 1.5605 - val_accuracy: 0.0078 - 11s/epoch - 235ms/step
Epoch 8/40
48/48 - 11s - loss: 0.8848 - accuracy: 0.7005 - val_loss: 1.5756 - val_accuracy: 0.0117 - 11s/epoch -

In [46]:
model.evaluate(test_dataset,test_dataset_label)

5/5 [==============================] - 1s 141ms/step - loss: 1.5441 - accuracy: 0.3187


[1.544090986251831, 0.3187499940395355]

In [47]:
model.evaluate(train_dataset,train_dataset_label)

20/20 [==============================] - 3s 144ms/step - loss: 0.8465 - accuracy: 0.6438


[0.8465191721916199, 0.643750011920929]

In [48]:
len(train_dataset)

640

In [53]:
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 20, 20, 20)]      0         
                                                                 
 conv2d_7 (Conv2D)           (None, 20, 20, 128)       23168     
                                                                 
 activation_5 (Activation)   (None, 20, 20, 128)       0         
                                                                 
 batch_normalization_9 (Batc  (None, 20, 20, 128)      512       
 hNormalization)                                                 
                                                                 
 conv2d_8 (Conv2D)           (None, 20, 20, 128)       16384     
                                                                 
 activation_6 (Activation)   (None, 20, 20, 128)       0         
                                                           

In [56]:
def createAndTrainResNetB1():
    
    learning_rate_base = LEARNING_RATE_BASE
    kernel_size = 3
    num_epochs = NUM_EPOCHS
    batch_size = BATCH_SIZE
    dropout_rate = 0.15 
    activation_type = 'relu'
    num_nodes_fc = 64
    wheat_types =  VARIETIES
    normalization_type = 'max'
    num_layers_each_block = [1, 0, 0, 0]
    num_chan_per_block = [32, 128, 256, 256]
    N_classes = len(wheat_types)
    
    ############ Load data ############
    print("--------------Load Data--------------")

    # Load training data and their corresponding labels
    x_training = np.array(train_dataset)
    labels_training = np.array(train_dataset_label)
    
    # Normalize the data
    x_training = normalizeDataWholeSeed(x_training,normalization_type=normalization_type)
    
    # Extract some information
    num_training = x_training.shape[0]
    N_spatial = x_training.shape[1:3]
    N_bands = x_training.shape[3]
    
    print('#training = %d' %(num_training))
    
    print("--------------Done--------------")
    
    
    ############ Prepare the path for saving the models/stats ############
#     print("--------------Prepare a path for saving the models/stats--------------")
    
#     hparams = make_hyperparam_string(learning_rate_base, batch_size, kernel_size, dropout_rate,
#                                      num_training, num_nodes_fc, activation_type)
#     print('Saving the model to...')
    
#     results_dir = os.path.join('./results/',hparams)
    
#     if not os.path.exists(results_dir):
#         os.makedirs(results_dir)
#     print(results_dir)

#     print("--------------Done--------------")

    ############ Create a model ############
    print("--------------Create a model--------------")
    
    # Generate a model
    model = ResNet2D_classifier(data_num_rows=N_spatial[0], data_num_cols=N_spatial[1], num_classes=N_classes,
                                kernel_size=kernel_size, num_layers_each_block=num_layers_each_block,
                                num_chan_per_block=num_chan_per_block, activation_type=activation_type,
                                dropout_rate=dropout_rate, num_input_chans=N_bands, num_nodes_fc=num_nodes_fc)

    # Compile the model
    adam_opt = Adam(learning_rate=LEARNING_RATE_BASE, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.01)
    model.compile(loss='sparse_categorical_crossentropy', optimizer=adam_opt, metrics=['accuracy'])

    # Create a Tensorboard callback
#     tbCallBack = TensorBoard(log_dir=results_dir, histogram_freq=0, write_graph=False, write_images=False)
    
    print("--------------Done--------------")

    ############ Train the model ############
    print("--------------Begin training the model--------------")

#     tic = timeit.default_timer()
    
    # Train the model
#     hist = model.fit(x=x_training,y=labels_training,batch_size=batch_size,  epochs = num_epochs, initial_epoch = 0, verbose=2, callbacks = [tbCallBack],validation_split=0.15,shuffle=True)

#     toc = timeit.default_timer()
#     training_time = toc-tic
#     print("Total training time = " + str(training_time))
    
    print("--------------Done--------------")

    return model

In [57]:
model1 = createAndTrainResNetB1()

--------------Load Data--------------
#training = 640
--------------Done--------------
--------------Create a model--------------
--------------Done--------------
--------------Begin training the model--------------
--------------Done--------------


In [59]:
tic = start_timer()
model1.fit(x=train_dataset,y=train_dataset_label,batch_size=BATCH_SIZE, epochs=20, initial_epoch = 0, verbose=2,validation_split=0.40,shuffle=True)
toc = end_timer()
show_time(tic,toc)

Testing started
Epoch 1/20
48/48 - 5s - loss: 1.2590 - accuracy: 0.4557 - val_loss: 1.4654 - val_accuracy: 0.0703 - 5s/epoch - 102ms/step
Epoch 2/20
48/48 - 5s - loss: 1.2556 - accuracy: 0.4714 - val_loss: 1.4692 - val_accuracy: 0.0508 - 5s/epoch - 102ms/step
Epoch 3/20
48/48 - 5s - loss: 1.2270 - accuracy: 0.5182 - val_loss: 1.4764 - val_accuracy: 0.0742 - 5s/epoch - 103ms/step
Epoch 4/20
48/48 - 5s - loss: 1.1877 - accuracy: 0.5443 - val_loss: 1.4760 - val_accuracy: 0.0898 - 5s/epoch - 102ms/step
Epoch 5/20
48/48 - 5s - loss: 1.1800 - accuracy: 0.5703 - val_loss: 1.4830 - val_accuracy: 0.0664 - 5s/epoch - 100ms/step
Epoch 6/20
48/48 - 5s - loss: 1.1706 - accuracy: 0.5573 - val_loss: 1.4885 - val_accuracy: 0.0898 - 5s/epoch - 102ms/step
Epoch 7/20
48/48 - 5s - loss: 1.1487 - accuracy: 0.5911 - val_loss: 1.4938 - val_accuracy: 0.0977 - 5s/epoch - 107ms/step
Epoch 8/20
48/48 - 5s - loss: 1.1483 - accuracy: 0.5964 - val_loss: 1.4997 - val_accuracy: 0.1016 - 5s/epoch - 106ms/step
Epoch 9/

In [60]:
model.evaluate(test_dataset,test_dataset_label)

5/5 [==============================] - 1s 223ms/step - loss: 1.5441 - accuracy: 0.3187


[1.544090986251831, 0.3187499940395355]

In [61]:
tic = start_timer()
model1.fit(x=train_dataset,y=train_dataset_label,batch_size=BATCH_SIZE, epochs=20, initial_epoch = 0, verbose=2,validation_split=0.40,shuffle=True)
toc = end_timer()
show_time(tic,toc)

Testing started
Epoch 1/20
48/48 - 5s - loss: 1.0177 - accuracy: 0.6953 - val_loss: 1.5328 - val_accuracy: 0.1055 - 5s/epoch - 108ms/step
Epoch 2/20
48/48 - 5s - loss: 1.0029 - accuracy: 0.7135 - val_loss: 1.5355 - val_accuracy: 0.1055 - 5s/epoch - 106ms/step
Epoch 3/20
48/48 - 5s - loss: 1.0018 - accuracy: 0.7161 - val_loss: 1.5381 - val_accuracy: 0.1016 - 5s/epoch - 107ms/step
Epoch 4/20
48/48 - 5s - loss: 1.0306 - accuracy: 0.6953 - val_loss: 1.5383 - val_accuracy: 0.1016 - 5s/epoch - 107ms/step
Epoch 5/20
48/48 - 5s - loss: 0.9886 - accuracy: 0.7057 - val_loss: 1.5374 - val_accuracy: 0.1055 - 5s/epoch - 107ms/step
Epoch 6/20
48/48 - 5s - loss: 0.9799 - accuracy: 0.7135 - val_loss: 1.5385 - val_accuracy: 0.1016 - 5s/epoch - 107ms/step
Epoch 7/20
48/48 - 5s - loss: 0.9653 - accuracy: 0.7240 - val_loss: 1.5419 - val_accuracy: 0.1016 - 5s/epoch - 107ms/step
Epoch 8/20
48/48 - 5s - loss: 0.9900 - accuracy: 0.7057 - val_loss: 1.5428 - val_accuracy: 0.1016 - 5s/epoch - 107ms/step
Epoch 9/

In [63]:
model1.evaluate(test_dataset,test_dataset_label)

5/5 [==============================] - 1s 43ms/step - loss: 1.3947 - accuracy: 0.3875


[1.3946599960327148, 0.38749998807907104]

In [64]:
model1.evaluate(train_dataset,train_dataset_label)

20/20 [==============================] - 1s 37ms/step - loss: 1.0871 - accuracy: 0.5953


[1.0870788097381592, 0.5953124761581421]

In [66]:
for x in range(4):
    print("epochs: ",40+2*x," to ",60+2*x)
    tic = start_timer()
    print(model1.fit(x=train_dataset,y=train_dataset_label,batch_size=BATCH_SIZE, epochs=20, initial_epoch = 0, verbose=2,validation_split=0.40,shuffle=True))
    toc = end_timer()
    show_time(tic,toc)
    print("for testing")
    print(model1.evaluate(test_dataset,test_dataset_label))
    print("for training")
    print(model1.evaluate(train_dataset,train_dataset_label))

epochs:  40  to  60
Testing started
Epoch 1/20
48/48 - 5s - loss: 0.9018 - accuracy: 0.7604 - val_loss: 1.5688 - val_accuracy: 0.0977 - 5s/epoch - 106ms/step
Epoch 2/20
48/48 - 5s - loss: 0.9001 - accuracy: 0.7786 - val_loss: 1.5700 - val_accuracy: 0.0977 - 5s/epoch - 106ms/step
Epoch 3/20
48/48 - 5s - loss: 0.9113 - accuracy: 0.7474 - val_loss: 1.5678 - val_accuracy: 0.0977 - 5s/epoch - 106ms/step
Epoch 4/20
48/48 - 5s - loss: 0.9367 - accuracy: 0.7630 - val_loss: 1.5711 - val_accuracy: 0.0977 - 5s/epoch - 107ms/step
Epoch 5/20
48/48 - 5s - loss: 0.9069 - accuracy: 0.7682 - val_loss: 1.5695 - val_accuracy: 0.0977 - 5s/epoch - 107ms/step
Epoch 6/20
48/48 - 5s - loss: 0.8803 - accuracy: 0.7917 - val_loss: 1.5755 - val_accuracy: 0.0977 - 5s/epoch - 106ms/step
Epoch 7/20
48/48 - 5s - loss: 0.8643 - accuracy: 0.7995 - val_loss: 1.5695 - val_accuracy: 0.0977 - 5s/epoch - 106ms/step
Epoch 8/20
48/48 - 5s - loss: 0.8820 - accuracy: 0.7786 - val_loss: 1.5775 - val_accuracy: 0.0977 - 5s/epoch -

Testing time (s) = 102.06342180003412

for testing
5/5 [==============================] - 0s 42ms/step - loss: 1.4126 - accuracy: 0.3812
[1.4126389026641846, 0.3812499940395355]
for training
20/20 [==============================] - 1s 41ms/step - loss: 1.0199 - accuracy: 0.6125
[1.0198969841003418, 0.612500011920929]
epochs:  46  to  66
Testing started
Epoch 1/20
48/48 - 5s - loss: 0.7809 - accuracy: 0.8203 - val_loss: 1.6423 - val_accuracy: 0.0859 - 5s/epoch - 108ms/step
Epoch 2/20
48/48 - 5s - loss: 0.7609 - accuracy: 0.8359 - val_loss: 1.6457 - val_accuracy: 0.0820 - 5s/epoch - 107ms/step
Epoch 3/20
48/48 - 5s - loss: 0.7507 - accuracy: 0.8411 - val_loss: 1.6517 - val_accuracy: 0.0820 - 5s/epoch - 103ms/step
Epoch 4/20
48/48 - 5s - loss: 0.7361 - accuracy: 0.8594 - val_loss: 1.6500 - val_accuracy: 0.0820 - 5s/epoch - 109ms/step
Epoch 5/20
48/48 - 5s - loss: 0.7685 - accuracy: 0.8333 - val_loss: 1.6541 - val_accuracy: 0.0820 - 5s/epoch - 110ms/step
Epoch 6/20
48/48 - 6s - loss: 0.744